In [1]:
import sqlite3
from mne.io import read_raw_edf
import pandas as pd
import os
import datetime
import torch
import numpy as np

edf_files = os.listdir(f'courtney_aug_oct_2022_baseline_recordings/1_raw_edf')
label_files = os.listdir(f'courtney_aug_oct_2022_baseline_recordings/2_labels')

ids = [f.replace('.edf','') for f in edf_files if any([f.replace('.edf','') in label for label in label_files])]

edf_files = [f'courtney_aug_oct_2022_baseline_recordings/1_raw_edf/{id}.edf' for id in ids]
label_files = [f'courtney_aug_oct_2022_baseline_recordings/2_labels/{id}.xls' for id in ids]

for edf,label in zip(edf_files,label_files):
    print(edf,label)

courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Oct-H.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Oct-H.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Aug-D.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Aug-D.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Oct-E.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Oct-E.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Oct-G.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Oct-G.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Aug-H.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Aug-H.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Aug-F.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Aug-F.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Aug-B.edf courtney_aug_oct_2022_baseline_recordings/2_labels/22-Aug-B.xls
courtney_aug_oct_2022_baseline_recordings/1_raw_edf/22-Oct-D.edf courtney_aug_oct_2022_baseline_r

In [2]:
for id in ids[:1]:
    print(id)
    try:
        conn = sqlite3.connect(f'courtney_aug_oct_2022_baseline_recordings/0_raw_zdb/{id}.zdb')
    except Exception as e:
        print(e)
    cur = conn.cursor()
    query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
    cur.execute(query)
    result = cur.fetchall()
    recording_start = int(result[0][0])
    query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
    cur.execute(query)
    result = cur.fetchall()
    recording_stop = int(result[0][0])
    length_ns = recording_stop - recording_start # ns
    length_s = length_ns * 1e-7 # s
    hh = length_s // 3600
    mm = (length_s % 3600) // 60
    ss = ((length_s % 3600) % 60)
    # print(hh,mm,ss,length_s)
    # print(recording_start)
    # print(recording_stop)

    fs = 500
    raw = read_raw_edf(f'courtney_aug_oct_2022_baseline_recordings/1_raw_edf/{id}.edf', 
                    verbose=False, 
                    preload=False,
                    include=['EEG 1','EEG 2'])
    data = raw.get_data()
    eeg,emg = data[0],data[1]
    measurement_date = raw.info["meas_date"]
    measurement_date = measurement_date.replace(tzinfo=None)
    edf_starting_timestamp = int((measurement_date - datetime.datetime(1, 1, 1)).total_seconds() * 10**7)

    query = "SELECT starts_at,type FROM scoring_marker"
    cur.execute(query)
    result = cur.fetchall()
    df = pd.DataFrame(result)
    df.loc[df[1] == 'Sleep-Wake',1] = 2
    df.loc[df[1] == 'Sleep-SWS',1] = 1
    df.loc[df[1] == 'Sleep-Paradoxical',1] = 0
    df = df.loc[~((df[1] != 2) & (df[1] != 1) & (df[1] != 0))]
    df.reset_index(drop=True,inplace=True)

    while True:
        scoring_starting_timestamp = df.iloc[0,0]
        print(scoring_starting_timestamp)
        scoring_offset_from_edf_starting_timestamp_seconds = int((scoring_starting_timestamp - edf_starting_timestamp)*1e-7)
        print(scoring_offset_from_edf_starting_timestamp_seconds)
        if scoring_offset_from_edf_starting_timestamp_seconds < 0:
            print('scoring less than 0')
            df = df.drop(0).reset_index(drop=True)
        else:
            break
        
    print(scoring_offset_from_edf_starting_timestamp_seconds)
    eeg = eeg[scoring_offset_from_edf_starting_timestamp_seconds*fs:]
    emg = emg[scoring_offset_from_edf_starting_timestamp_seconds*fs:]

    if len(df) > eeg.shape[0] / fs / 10:
        if eeg.shape[0] % 5000 != 0:
            eeg = np.concatenate([eeg,np.zeros((5000 - eeg.shape[0] % 5000))])
            emg = np.concatenate([emg,np.zeros((5000 - emg.shape[0] % 5000))])
    else:
        eeg = eeg[:len(df)*5000]
        emg = emg[:len(df)*5000]

    eeg = eeg.reshape(-1,5000)
    emg = emg.reshape(-1,5000)

    eeg = torch.from_numpy(eeg).float()
    emg = torch.from_numpy(emg).float()
    y = torch.nn.functional.one_hot(torch.from_numpy(df[1].values.astype('int')),num_classes=3).float()

    if len(eeg) < len(df):
        y = y[:len(eeg)]

    print(eeg.shape,emg.shape,y.shape)

    torch.save((eeg,emg,y),f'pt_courtney_eeg_emg/{id}.pt')

22-Oct-H
638024801200000000
32
32
torch.Size([41399, 5000]) torch.Size([41399, 5000]) torch.Size([41399, 3])


In [ ]:
df